In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import csv as csv
import numpy as np
import geocoder

In [2]:
df = pd.read_csv(r'C:\Users\gbier\Documents\Real_Estate\_All_Real_Estate_Locations.csv')

In [3]:
df

,Unnamed: 0,Price,Building_Type,Rooms,Address,City,Latitude,Longitude
0,0,"$649,900",House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161
1,1,"$500,000",House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162
2,2,"$499,900",House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996
3,3,"$499,900",Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638
4,4,"$479,900",House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532
...,...,...,...,...,...,...,...,...
12953,12953,"$1,095,000",House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,"76.485672"""
12954,12954,"$624,900",House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,-76.589800
12955,12955,"$599,900",House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144
12956,12956,"$524,900",Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,"44.234139""","6.479333"">"


In [4]:
# initial clean up to do
# get rid of unamed column
df.pop("Unnamed: 0")
# get rid of $ and , in price column
df['Price'] = df['Price'].apply(lambda x: x.replace('$', ''))
df['Price'] = df['Price'].apply(lambda x: x.replace(',', ''))
#get rid of weird symbols in longitude and latitude
df['Longitude'] = df['Longitude'].apply(lambda x: x.replace('-', ''))
df['Longitude'] = df['Longitude'].apply(lambda x: x.replace('"', ''))
df['Longitude'] = df['Longitude'].apply(lambda x: x.replace('>', ''))
df['Latitude'] = df['Latitude'].apply(lambda x: x.replace('"', ''))

In [5]:
df

,Price,Building_Type,Rooms,Address,City,Latitude,Longitude
0,649900,House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161
1,500000,House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162
2,499900,House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996
3,499900,Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638
4,479900,House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532
...,...,...,...,...,...,...,...
12953,1095000,House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,76.485672
12954,624900,House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,76.589800
12955,599900,House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144
12956,524900,Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,44.234139,6.479333


In [6]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))

In [7]:
# we can see that 9% of column has missing values - not bad
missing(df)

Rooms            9.0
Longitude        0.0
Latitude         0.0
City             0.0
Address          0.0
Building_Type    0.0
Price            0.0
dtype: float64


In [8]:
#First replace any blank with np.nan
df['Rooms'].replace(' ', np.nan, inplace=True)

In [9]:
#drop na
df.dropna(subset=['Rooms'], inplace=True)

In [10]:
df

,Price,Building_Type,Rooms,Address,City,Latitude,Longitude
0,649900,House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161
1,500000,House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162
2,499900,House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996
3,499900,Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638
4,479900,House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532
...,...,...,...,...,...,...,...
12953,1095000,House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,76.485672
12954,624900,House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,76.589800
12955,599900,House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144
12956,524900,Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,44.234139,6.479333


In [11]:
# we can still see bad values in the rooms solumns lets get rid of these
sorted_duplicates = df['Rooms'].drop_duplicates()
sorted_duplicates

0          6 bds , 3 bths
1          3 bds , 3 bths
2          5 bds , 3 bths
3          2 bds , 3 bths
4          3 bds , 2 bths
6          5 bds , 2 bths
7          4 bds , 2 bths
8          3 bds , 1 bath
9          2 bds , 1 bath
14         4 bds , 3 bths
21         2 bds , 2 bths
28         1 bed , 1 bath
36                 1 bath
905        3 bds , 5 bths
909        3 bds , 4 bths
912        6 bds , 4 bths
930        4 bds , 4 bths
932        5 bds , 4 bths
1925       7 bds , 4 bths
1941       8 bds , 3 bths
2931       4 bds , 1 bath
2965       5 bds , 5 bths
3941       4 bds , 5 bths
3942       6 bds , 5 bths
3943       8 bds , 5 bths
3944       6 bds , 6 bths
3951       7 bds , 3 bths
3958     12 bds , 10 bths
3971       7 bds , 2 bths
4953       7 bds , 5 bths
9010       6 bds , 2 bths
9997           $1,075,000
11020          $2,499,000
11990    11 bds , 11 bths
Name: Rooms, dtype: object

In [12]:
df['Rooms'] = df['Rooms'].apply(lambda x: x.replace('$1,075,000', ''))
df['Rooms'] = df['Rooms'].apply(lambda x: x.replace('$2,499,000', ''))
df['Rooms'].replace('', np.nan, inplace=True)
df.dropna(subset=['Rooms'], inplace=True)
sorted_duplicates_two = df['Rooms'].drop_duplicates()
sorted_duplicates_two

0          6 bds , 3 bths
1          3 bds , 3 bths
2          5 bds , 3 bths
3          2 bds , 3 bths
4          3 bds , 2 bths
6          5 bds , 2 bths
7          4 bds , 2 bths
8          3 bds , 1 bath
9          2 bds , 1 bath
14         4 bds , 3 bths
21         2 bds , 2 bths
28         1 bed , 1 bath
36                 1 bath
905        3 bds , 5 bths
909        3 bds , 4 bths
912        6 bds , 4 bths
930        4 bds , 4 bths
932        5 bds , 4 bths
1925       7 bds , 4 bths
1941       8 bds , 3 bths
2931       4 bds , 1 bath
2965       5 bds , 5 bths
3941       4 bds , 5 bths
3942       6 bds , 5 bths
3943       8 bds , 5 bths
3944       6 bds , 6 bths
3951       7 bds , 3 bths
3958     12 bds , 10 bths
3971       7 bds , 2 bths
4953       7 bds , 5 bths
9010       6 bds , 2 bths
11990    11 bds , 11 bths
Name: Rooms, dtype: object

In [13]:
df['RoomCount'] = df['Rooms'].str.split("").str.len()

In [14]:
df

,Price,Building_Type,Rooms,Address,City,Latitude,Longitude,RoomCount
0,649900,House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161,16
1,500000,House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162,16
2,499900,House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996,16
3,499900,Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638,16
4,479900,House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532,16
...,...,...,...,...,...,...,...,...
12953,1095000,House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,76.485672,16
12954,624900,House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,76.589800,16
12955,599900,House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144,16
12956,524900,Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,44.234139,6.479333,16


In [15]:
df = df[df['RoomCount'] > 8]  

In [16]:
sorted_duplicates_four = df['Rooms'].drop_duplicates()
sorted_duplicates_four

0          6 bds , 3 bths
1          3 bds , 3 bths
2          5 bds , 3 bths
3          2 bds , 3 bths
4          3 bds , 2 bths
6          5 bds , 2 bths
7          4 bds , 2 bths
8          3 bds , 1 bath
9          2 bds , 1 bath
14         4 bds , 3 bths
21         2 bds , 2 bths
28         1 bed , 1 bath
905        3 bds , 5 bths
909        3 bds , 4 bths
912        6 bds , 4 bths
930        4 bds , 4 bths
932        5 bds , 4 bths
1925       7 bds , 4 bths
1941       8 bds , 3 bths
2931       4 bds , 1 bath
2965       5 bds , 5 bths
3941       4 bds , 5 bths
3942       6 bds , 5 bths
3943       8 bds , 5 bths
3944       6 bds , 6 bths
3951       7 bds , 3 bths
3958     12 bds , 10 bths
3971       7 bds , 2 bths
4953       7 bds , 5 bths
9010       6 bds , 2 bths
11990    11 bds , 11 bths
Name: Rooms, dtype: object

In [17]:
df.drop(['RoomCount'], inplace=True, axis=1)
df

C:\Users\gbier\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Price,Building_Type,Rooms,Address,City,Latitude,Longitude
0,649900,House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161
1,500000,House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162
2,499900,House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996
3,499900,Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638
4,479900,House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532
...,...,...,...,...,...,...,...
12953,1095000,House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,76.485672
12954,624900,House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,76.589800
12955,599900,House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144
12956,524900,Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,44.234139,6.479333


In [18]:
df[['Beds','Baths']] = df['Rooms'].str.split(',',expand=True)
df

C:\Users\gbier\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Price,Building_Type,Rooms,Address,City,Latitude,Longitude,Beds,Baths
0,649900,House,"6 bds , 3 bths",412 Church St,Windsor,42.3152526,83.0421161,6 bds,3 bths
1,500000,House,"3 bds , 3 bths",1143 Hickory,Windsor,42.3189102,82.9997162,3 bds,3 bths
2,499900,House,"5 bds , 3 bths",3196 Avondale Avenue,Windsor,42.2677498,83.0138996,5 bds,3 bths
3,499900,Condo,"2 bds , 3 bths",5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638,2 bds,3 bths
4,479900,House,"3 bds , 2 bths",1054 Ivanhill,Windsor,42.3258351,82.9145532,3 bds,2 bths
...,...,...,...,...,...,...,...,...,...
12953,1095000,House,"3 bds , 2 bths",91 Gore St,Kingston,44.2271809,76.485672,3 bds,2 bths
12954,624900,House,"4 bds , 3 bths",268 Holden Street,Kingston,44.2656851,76.589800,4 bds,3 bths
12955,599900,House,"5 bds , 3 bths",778 Brock St,Kingston,44.2321918,76.5118144,5 bds,3 bths
12956,524900,Townhouse,"3 bds , 3 bths",1 Place D'Armes # 52,Kingston,44.234139,6.479333,3 bds,3 bths


In [19]:

sorted_duplicates_five = df['Baths'].drop_duplicates()
sorted_duplicates_five

0          3 bths
4          2 bths
8          1 bath
905        5 bths
909        4 bths
3944       6 bths
3958      10 bths
11990     11 bths
Name: Baths, dtype: object

In [20]:
df['Beds'] = df['Beds'].apply(lambda x: x.replace(' bds', ''))
df['Baths'] = df['Baths'].apply(lambda x: x.replace(' bths', ''))
df['Baths'] = df['Baths'].apply(lambda x: x.replace(' bath', ''))
df['Beds'] = df['Beds'].apply(lambda x: x.replace(' bed ', ''))
df.drop(['Rooms'], inplace=True, axis=1)
df

<ipython-input-20-8f488df27ee6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Beds'] = df['Beds'].apply(lambda x: x.replace(' bds', ''))
<ipython-input-20-8f488df27ee6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Baths'] = df['Baths'].apply(lambda x: x.replace(' bths', ''))
<ipython-input-20-8f488df27ee6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,Price,Building_Type,Address,City,Latitude,Longitude,Beds,Baths
0,649900,House,412 Church St,Windsor,42.3152526,83.0421161,6,3
1,500000,House,1143 Hickory,Windsor,42.3189102,82.9997162,3,3
2,499900,House,3196 Avondale Avenue,Windsor,42.2677498,83.0138996,5,3
3,499900,Condo,5955 Ontario Unit# 105,Windsor,42.3271511,82.9645638,2,3
4,479900,House,1054 Ivanhill,Windsor,42.3258351,82.9145532,3,2
...,...,...,...,...,...,...,...,...
12953,1095000,House,91 Gore St,Kingston,44.2271809,76.485672,3,2
12954,624900,House,268 Holden Street,Kingston,44.2656851,76.589800,4,3
12955,599900,House,778 Brock St,Kingston,44.2321918,76.5118144,5,3
12956,524900,Townhouse,1 Place D'Armes # 52,Kingston,44.234139,6.479333,3,3


In [21]:

sorted_duplicates_five = df['Beds'].drop_duplicates()
sorted_duplicates_five

0         6 
1         3 
2         5 
3         2 
7         4 
28         1
1925      7 
1941      8 
3958     12 
11990    11 
Name: Beds, dtype: object

In [22]:
# back to csv for tableau analysis
df.to_csv(r'C:\Users\gbier\Documents\Real_Estate\_All_Real_Estate_Fixed.csv')